# Support Vector Machines

Have a look at [Chapter 5.07 of PDSH on Support Vector Machines](https://jakevdp.github.io/PythonDataScienceHandbook/05.07-support-vector-machines.html)[![Cover image of PDSH](images/PDSH-cover-small.png)](https://jakevdp.github.io/PythonDataScienceHandbook/05.07-support-vector-machines.html)

SVMs are another classification method. Here's the example from PDSH, modified to use plotnine:




In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
from scipy import stats

from sklearn.datasets import make_blobs, make_circles
from sklearn.datasets import fetch_lfw_people

from sklearn.svm import SVC # "Support vector classifier"
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split



# I didn't change all the plotting code to plotnine
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
from mpl_toolkits import mplot3d

# I did change some plots to use plotnine
import plotnine as pn

# Creat some data:
#   X is a 2D-ndarray with x,y coordinates
#   y is 1/0 categorical value for each X
X, y = make_blobs(n_samples=50, centers=2,
                  random_state=0, cluster_std=0.60)

# Convert to dataframe for pn
df=pd.DataFrame({'x':X[:,0], 'y':X[:,1], 'cat':y } )

pn.ggplot(df, pn.aes(x='x', y='y')) + pn.geom_point(pn.aes(color='factor(df.cat)'))

> A linear discriminative classifier would attempt to draw a straight line separating the two sets of data, and thereby create a model for classification. For two dimensional data like that shown here, this is a task we could do by hand. But immediately we see a problem: there is more than one possible dividing line that can perfectly discriminate between the two classes!

> We can draw them as follows:

In [ ]:
pn.ggplot(df, pn.aes(x='x', y='y')) + pn.geom_point(pn.aes(color='factor(df.cat)')) + pn.geom_abline(pn.aes(slope=1, intercept=0.65))+ pn.geom_abline(pn.aes(slope=0.5, intercept=1.6))+ pn.geom_abline(pn.aes(slope=-0.2, intercept=2.9)) + pn.geom_point(pn.aes(x=0.6, y=2.1), color='red', shape='x', size=5)

> These are three very different separators which, nevertheless, perfectly discriminate between these samples. Depending on which you choose, a new data point (e.g., the one marked by the "X" in this plot) will be assigned a different label! Evidently our simple intuition of "drawing a line between classes" is not enough, and we need to think a bit deeper.

With the Support Vector Machine algorithm, the objective is to find a line (in the case of 2D, or **hyperplane** in N-dimensional space) that has the **maximum margin**, or distance between datapoints of each class. 

The SVM options include a `kernel`--we'll use `linear` for now and come back to this later--and `C`. $C$ is the cost parameter that controls the width of the margin, or how costly is it to misclassify something.

In [ ]:
# This is the plotting function from PDSH, I didn't try to
# rewrite it using plotnine, so it uses matplotlib

def plot_svc_decision_function(model, ax=None, plot_support=True):
    """Plot the decision function for a 2D SVC"""
    if ax is None:
        ax = plt.gca()
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    
    # create grid to evaluate model
    x = np.linspace(xlim[0], xlim[1], 30)
    y = np.linspace(ylim[0], ylim[1], 30)
    Y, X = np.meshgrid(y, x)
    xy = np.vstack([X.ravel(), Y.ravel()]).T
    P = model['SVM'].decision_function(xy).reshape(X.shape)
    
    # plot decision boundary and margins
    ax.contour(X, Y, P, colors='k',
               levels=[-1, 0, 1], alpha=0.5,
               linestyles=['--', '-', '--'])
    
    # plot support vectors
    if plot_support:
        ax.scatter(model['SVM'].support_vectors_[:, 0],
                   model['SVM'].support_vectors_[:, 1],
                   s=300, linewidth=1, edgecolor='black', facecolors='none')
    ax.set_xlim(xlim)
    ax.set_ylim(ylim)

In [ ]:
plt.scatter(df.x, df.y, c=df.cat, s=50, cmap='autumn')
plot_svc_decision_function(model)

> This is the dividing line that maximizes the margin between the two sets of points. Notice that a few of the training points just touch the margin: they are indicated by the black circles in this figure. These points are the pivotal elements of this fit, and are known as the support vectors, and give the algorithm its name.

One way to think of the support vectors is that they are the critical elements of the training set and those which would change the position of the dividing hyperplane if removed.

Equally important is the fact that these are the **only** point that matter for the model. Removing other points has no impact on the model because they have are not part of the **loss function** used in SVMs.

In Scikit-Learn, the identity of these points are stored in the `support_vectors_` attribute of the classifier:

The graph above and the image on the left below show the line in a 2D example, while the graph on the right shows the plane (hyperplane) from a 3D feature space. It gets hard to draw after that... (image from Rohith Gandhi's [Support Vector Machine — Introduction to Machine Learning Algorithms](https://towardsdatascience.com/support-vector-machine-introduction-to-machine-learning-algorithms-934a444fca47))

![Hyperplanes in 2D and 3D feature space from Rohith Gandhi](images/SVM_hyperplane.png)

## The Hinge Loss Function

Without getting into the math, SVMs make use of a loss function called **hinge loss** (If you want more details, there are many sources online, though I found [Clare Liu's post here](https://www.vebuso.com/2020/02/a-top-machine-learning-algorithm-explained-support-vector-machines-svms/) particularly easy to follow and not too mathematical) Hinge loss penalizes misclassified points more the further they are from the decision boundary. As such, a point close to the hyperplane splitting classes will be penalized less than a point further away from the line. As with other loss functions, the idea is to sum the loss over all the training data and minimize the loss.


## The Kernel "Trick" and Mapping to a Higher Dimension

Another characteristic of SVMs is their use of kernel functions, also sometimes called the 'kernel trick'. For data that are not linearly separable, as above, SVMs can use a kernel function to transform the data to a higher dimension to make it linearly separable.

In [ ]:
# See PDSH Ch 5.07: Beyond linear boundaries: Kernel SVM

r = np.exp(-(X ** 2).sum(1)) # Apply radial bias function (rbf)

def plot_3D(elev=30, azim=30, X=X, y=y):
    ax = plt.subplot(projection='3d')
    ax.scatter3D(X[:, 0], X[:, 1], r, c=y, s=50, cmap='autumn')
    ax.view_init(elev=elev, azim=azim)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('r')

interact(plot_3D, elev=[-30, 30,-45, 45], azip=(-180, 180),
         X=fixed(X), y=fixed(y))

> Here we had to choose and carefully tune our projection: if we had not centered our radial basis function in the right location, we would not have seen such clean, linearly separable results. In general, the need to make such a choice is a problem: we would like to somehow automatically find the best basis functions to use.

> One strategy to this end is to compute a basis function centered at every point in the dataset, and let the SVM algorithm sift through the results. This type of basis function transformation is known as a kernel transformation, as it is based on a similarity relationship (or kernel) between each pair of points.

> A potential problem with this strategy—projecting $N$ points into $N$ dimensions—is that it might become very computationally intensive as $N$ grows large. However, because of a neat little procedure known as the [kernel trick](https://en.wikipedia.org/wiki/Kernel_method), a fit on kernel-transformed data can be done implicitly—that is, without ever building the full $N$-dimensional representation of the kernel projection! This kernel trick is built into the SVM, and is one of the reasons the method is so powerful.

## Tuning the SVM: Softening Margins

So far, we have looked at fairly easily separable classes. We also briefly mentioned $C$, but didn't really look at the effect of changing $C$.

When data have more overlap, we can use the SVMs fudge-factor to soften the margin, allowing some points to be within the margin to allow better fit. For very large values of $C$, the margin is hard. For smaller values of $C$, the margin is softer. I've added the 1E10 value used above to what the PDSH text shows.


In [ ]:
X, y = make_blobs(n_samples=100, centers=2,
                  random_state=0, cluster_std=0.8)

fig, ax = plt.subplots(1, 3, figsize=(16, 6))
fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.1)

for axi, C in zip(ax, [1E10, 10.0, 0.1]):
    model = Pipeline([('SVM', SVC(kernel='linear', C=C))])
    model.fit(X, y)
    axi.scatter(X[:, 0], X[:, 1], c=y, s=50, cmap='autumn')
    plot_svc_decision_function(model, axi)
    axi.scatter(model['SVM'].support_vectors_[:, 0],
                model['SVM'].support_vectors_[:, 1],
                s=300, lw=1, facecolors='none');
    axi.set_title('C = {0:.1f}'.format(C), size=14)

## Facial Recognition with SVMs

One area where SVMs are employed is with facial recognition. Again, from PDSH Ch 5.07:
>  We will use the Labeled Faces in the Wild dataset, which consists of several thousand collated photos of various public figures. A fetcher for the dataset is built into Scikit-Learn:

In [ ]:
# Get some face photos
faces = fetch_lfw_people(min_faces_per_person=60)
print(faces.target_names)
print(faces.images.shape)
print(type(faces))

In [ ]:
# Plot some faces
fig, ax = plt.subplots(3, 5)
for i, axi in enumerate(ax.flat):
    axi.imshow(faces.images[i], cmap='bone')
    axi.set(xticks=[], yticks=[],
            xlabel=faces.target_names[faces.target[i]])

> Each image contains 62×47 or nearly 3,000 pixels. We could proceed by simply using each pixel value as a feature, but often it is more effective to use some sort of preprocessor to extract more meaningful features; here we will use a principal component analysis (see [In Depth: Principal Component Analysis](https://jakevdp.github.io/PythonDataScienceHandbook/05.09-principal-component-analysis.html)) to extract 150 fundamental components to feed into our support vector machine classifier. We can do this most straightforwardly by packaging the preprocessor and the classifier into a single pipeline:

In [ ]:
# I updated to use Pipeline instead of the text's make_pipeline


In [ ]:
# Split the dataset into training and testing datasets


We kind of glossed over the bit above about how we "we had to choose and carefully tune our projection" and the same for the $C$ cost value. One way of figuring these values out is to test a bunch and see what works...

Luckily there is a `GridSearch` (and `GridScearchCV` that also does cross validation) that will test a set of parameters for us automatically.

In [ ]:
fig, ax = plt.subplots(4, 6)
for i, axi in enumerate(ax.flat):
    axi.imshow(Xtest[i].reshape(62, 47), cmap='bone')
    axi.set(xticks=[], yticks=[])
    axi.set_ylabel(faces.target_names[yfit[i]].split()[-1],
                   color='black' if yfit[i] == ytest[i] else 'red')
fig.suptitle('Predicted Names; Incorrect Labels in Red', size=14);

In [ ]:


# Plot the confusion matrix
cm_display = ConfusionMatrixDisplay(cm, display_labels=faces.target_names).plot(xticks_rotation='vertical')